# 🎁 [Day 2](https://adventofcode.com/2019/day/2)

In [1]:
def run_program(p):
    op = 0
    while p[op] != 99:
        ix, iy, dst = p[op + 1:op + 4]
        result = p[ix] + p[iy] if p[op] == 1 else p[ix] * p[iy]
        p[dst] = result
        op += 4
        
def alarm_1202(program):
    p = [x for x in program]
    p[1] = 12
    p[2] = 2
    run_program(p)
    return p[0]

In [2]:
with open("inputs/day2.txt", 'r') as f:
    inputs = list(map(int, f.read().split(',')))
    
print('1202 program alarm:', alarm_1202(inputs))

1202 program alarm: 3409710


For the second part, first do some reverse engineering to see what the code is doing. It seems that the second output (verb), only impact the result of the program as a linear factor (`ar[131] <- X2 + ar[127]`)

Therefore we do not need to loop over every value, as the relationship is pretty simple. On the other hand, the first value has a more nested impact as it modify pointers too.

In [3]:
def find_latest(p, ops, dst, shift=0):
    for i, (j, op, ix, iy, d) in enumerate(ops[::-1]):
        if d == dst:
            ops_fst_param = find_latest(p, ops[:-i-1], j + 1, shift + 1)   
            if len(ops_fst_param) == 0:
                ops_fst_src = find_latest(p, ops[:-i-1], ix, shift + 1)
                if len(ops_fst_src) == 0:           
                    if ix == 1:
                        x = "\033[1mnoun\033[0m"
                    elif ix == 2:
                        x = "\033[1mverb\033[0m"
                    else: 
                        x = str(p[ix])   
                else:
                    x = "ar[{}]".format(ix)
            else:
                ops_fst_src = []
                x = "ar[ar[{}]]".format(j + 1)
                
            ops_snd_param = find_latest(p, ops[:-i-1], j + 2, shift  +1)
            if len(ops_snd_param) == 0:
                ops_snd_src = find_latest(p, ops[:-i-1], iy, shift + 1)
                if len(ops_snd_src) == 0:   
                    if iy == 1:
                        y = "\033[1mnoun\033[0m"
                    elif iy == 2:
                        y = "\033[1mverb\033[0m"
                    else: 
                        y = str(p[iy]) 
                else:
                    y = "ar[{}]".format(iy)
            else:
                ops_snd_src = []
                y = "ar[ar[{}]]".format(j + 2)
                
            s = "{}({}): ar[{}] <- {} {} {}".format(
                ' ' * shift, j, d, x, '+' if op == 1 else 'x', y)
            return ops_fst_param + ops_fst_src + ops_snd_param + ops_snd_src + [(j, s)]
    return []

def reverse_engineer(program):
    p = [x for x in program]
    
    # Parse the program
    ops = []
    op = 0
    while p[op] != 99:
        ix, iy, dst = p[op + 1:op + 4]
        ops.append((op // 4, p[op], ix, iy, dst))
        op += 4
        
    # Remove duplicate operations
    strings = find_latest(p, ops, 0)
    seen = []
    for i, s in strings:
        if not i in seen:
            print(s)
            seen.append(i)
    
reverse_engineer(inputs)

       (4): ar[19] <- 5 x noun
      (5): ar[23] <- ar[19] + 3
     (6): ar[27] <- 1 + ar[23]
    (7): ar[31] <- ar[27] + 3
        (25): ar[103] <- 3 + ar[ar[27]]
       (26): ar[107] <- ar[ar[27]] + ar[103]
      (27): ar[111] <- 3 x ar[107]
     (28): ar[115] <- ar[111] + 1
    (29): ar[119] <- ar[115] + ar[ar[31]]
   (30): ar[123] <- ar[ar[31]] + ar[119]
  (31): ar[127] <- 5 + ar[123]
 (32): ar[131] <- verb + ar[127]
 (8): ar[35] <- 2 + ar[31]
(33): ar[0] <- ar[131] + ar[ar[35]]


In [4]:
def run(program, rng=99, target=19690720):
    for x1 in range(rng):
        p = [x for x in program]  
        p[2] = 0
        p[1] = x1
        run_program(p)
        if 0 <= (target - p[0]) <= rng:
            x2 = target - p[0]
            return 100 * x1 + x2
    
            
print("Target noun and verb:", run(inputs))

Target noun and verb: 7912
